In [3]:
!nvidia-smi

Mon Feb 10 15:29:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 430.50       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
| 58%   84C    P2   189W / 250W |  10893MiB / 11178MiB |     83%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0  On |                  N/A |
| 57%   83C    P2   162W / 250W |  10905MiB / 11175MiB |     78%      Default |
+-------------------------------+----------------------+----------------------+
        

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

from tensorflow.keras.callbacks import Callback

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

In [2]:
val_data_dir = '/opt/AIStorage/PLAYGROUND/data/validation'
val_data_dir = pathlib.Path(val_data_dir)

data_dir = '/opt/AIStorage/PLAYGROUND/data/train'
data_dir = pathlib.Path(data_dir)

In [3]:
image_count = len(list(data_dir.glob('*/*.png')))
image_count

3196

In [4]:
val_image_count = len(list(val_data_dir.glob('*/*.png')))
val_image_count

798

In [5]:
val_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [6]:
BATCH_SIZE = 8
IMG_HEIGHT = 512
IMG_WIDTH = 512
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.ceil(val_image_count/BATCH_SIZE)

In [7]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

validation_data_gen = val_image_generator.flow_from_directory(directory=str(val_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

Found 3196 images belonging to 2 classes.
Found 798 images belonging to 2 classes.


In [8]:
epochs = 6
lr = 3e-3
optimizer = tf.keras.optimizers.SGD(lr=lr)


In [9]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
base_model = tf.keras.applications.DenseNet169(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [10]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

batch_norm_1 = tf.keras.layers.BatchNormalization()
drop_out_1 = tf.keras.layers.Dropout(0.25)
dense_layer_1 = tf.keras.layers.Dense(512,activation='relu')

batch_norm_2 = tf.keras.layers.BatchNormalization()
drop_out_2 = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(2)

model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      batch_norm_1,
      drop_out_1,
      dense_layer_1,
      batch_norm_2,
      drop_out_2,
      prediction_layer
])

    
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
history = model.fit_generator(train_data_gen,
                    epochs=epochs,
                    validation_data=validation_data_gen, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_steps=VAL_STEPS_PER_EPOCH)

Epoch 1/6
400/400 [==============================] - 324s 810ms/step - loss: 0.8742 - accuracy: 0.5842 - val_loss: 1.1869 - val_accuracy: 0.5351
Epoch 2/6
400/400 [==============================] - 316s 791ms/step - loss: 0.7461 - accuracy: 0.6263 - val_loss: 1.8790 - val_accuracy: 0.5169
Epoch 3/6
400/400 [==============================] - 318s 794ms/step - loss: 0.6990 - accuracy: 0.6482 - val_loss: 1.2573 - val_accuracy: 0.5288
Epoch 4/6
400/400 [==============================] - 316s 790ms/step - loss: 0.6623 - accuracy: 0.6561 - val_loss: 1.4428 - val_accuracy: 0.5244
Epoch 5/6
259/400 [==================>...........] - ETA: 1:29 - loss: 0.6169 - accuracy: 0.6736

KeyboardInterrupt: 